# Predict Credit Card Fraud


In [1]:
import os
os.chdir(r'C:\Users\Pedram\Documents\GitHub\Credit_Card_Fraud_Prediction')

The data on 1000 simulated credit card transactions is found in the `file transactions_modified.csv`. We'll start by loading this data into a pandas DataFrame named `transactions`.

In [2]:
import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Load the data
transactions = pd.read_csv('transaction_modified.csv')
print(transactions.head())
print(transactions.info())

   step      type      amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0   206  CASH_OUT    62927.08   C473782114           0.00            0.00   
1   380   PAYMENT    32851.57  C1915112886           0.00            0.00   
2   570  CASH_OUT  1131750.38  C1396198422     1131750.38            0.00   
3   184  CASH_OUT    60519.74   C982551468       60519.74            0.00   
4   162   CASH_IN    46716.01  C1759889425     7668050.60      7714766.61   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isPayment  \
0  C2096898696       649420.67       712347.75        0          0   
1   M916879292            0.00            0.00        0          1   
2  C1612235515       313070.53      1444820.92        1          0   
3  C1378644910        54295.32       182654.50        1          0   
4  C2059152908      2125468.75      2078752.75        0          0   

   isMovement  accountDiff  
0           1    649420.67  
1           0         0.00  
2           1    818679.85  


In [3]:
# How many fraudulent transactions?
print('The num of fraudulent transactions: ', transactions.isFraud.sum())

The num of fraudulent transactions:  282


2. Summary statistics of `amount` column to have a general view of the distribution.

In [4]:
# Summary statistics on amount column
transactions['amount'].describe()

count    1.000000e+03
mean     5.373080e+05
std      1.423692e+06
min      0.000000e+00
25%      2.933705e+04
50%      1.265305e+05
75%      3.010378e+05
max      1.000000e+07
Name: amount, dtype: float64

3. A new column named `isPayment` will be introduced. This column will be assigned a value of `1` in cases where the type is designated as either "PAYMENT" or "DEBIT," and a value of `0` in other instances.

In [5]:
transactions['isPayment'] = 0
transactions.loc[transactions['type'].isin(['PAYMENT', 'DEBIT']), 'isPayment'] = 1

In [6]:
payment = transactions[transactions.type == 'PAYMENT']
payment.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isPayment,isMovement,accountDiff
count,214.000000,214.000000,214.000000,214.000000,214.0,214.0,214.0,214.0,214.0,214.000000
mean,247.906542,13442.746308,52711.049720,46573.490701,0.0,0.0,0.0,1.0,0.0,52711.049720
std,127.425429,14006.458934,105727.989882,102165.163351,0.0,0.0,0.0,0.0,0.0,105727.989882
min,1.000000,206.690000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.000000
25%,177.250000,3442.995000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.000000
50%,252.500000,8719.545000,3493.000000,0.000000,0.0,0.0,0.0,1.0,0.0,3493.000000
75%,331.750000,19097.352500,52815.672500,44299.232500,0.0,0.0,0.0,1.0,0.0,52815.672500
max,596.000000,76894.320000,610962.000000,603792.350000,0.0,0.0,0.0,1.0,0.0,610962.000000


In [7]:
payment = transactions[transactions.type == 'DEBIT']
payment.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isPayment,isMovement,accountDiff
count,6.000000,6.000000,6.000000,6.000000,6.000000e+00,6.000000e+00,6.0,6.0,6.0,6.000000e+00
mean,213.000000,11848.195000,48101.151667,37769.358333,1.196419e+06,1.208268e+06,0.0,1.0,0.0,1.148318e+06
std,132.152942,20965.386989,41091.540004,35222.822343,1.782576e+06,1.776041e+06,0.0,0.0,0.0,1.788929e+06
min,11.000000,449.190000,0.000000,0.000000,5.766194e+04,6.676035e+04,0.0,1.0,0.0,5.766194e+04
25%,144.750000,1597.637500,28783.250000,26032.195000,1.592953e+05,1.753036e+05,0.0,1.0,0.0,9.018931e+04
50%,217.500000,3096.445000,31489.500000,29453.945000,4.461907e+05,4.735098e+05,0.0,1.0,0.0,3.723600e+05
75%,325.500000,7644.182500,77329.682500,35960.115000,1.218794e+06,1.219776e+06,0.0,1.0,0.0,1.170148e+06
max,350.000000,54188.960000,105137.000000,104687.810000,4.688481e+06,4.691393e+06,0.0,1.0,0.0,4.655318e+06


In [8]:
#double checking if the functions worked well
transactions.isPayment.sum()
#214 + 6 = 220

220

4. Likewise, a column named `isMovement` should be generated to show whether funds shifted away from the source account. In this column, a value of `1` will be assigned if the type is either "CASH_OUT" or "TRANSFER," and a value of `0` otherwise.

In [9]:
transactions['isMovement'] = 0
transactions.loc[transactions['type'].isin(['CASH_OUT', 'TRANSFER']), 'isMovement'] = 1

In [10]:
transactions.isMovement.sum()

605

5. In the investigation of financial fraud, an additional crucial aspect to consider would involve the distinction in value between the source and destination accounts. In this instance, our theory suggests that destination accounts showing a notably distinct value could raise suspicions of potential fraud. To proceed, we will establish a column termed `accountDiff`, which will contain the absolute difference between the values in the `oldbalanceOrg` and `oldbalanceDest` columns.

In [11]:
# Create accountDiff field
transactions['accountDiff'] = abs(transactions['oldbalanceDest'] - transactions['oldbalanceOrg'])

print(transactions.columns)

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isPayment',
       'isMovement', 'accountDiff'],
      dtype='object')


##### Feature selection, labeling, train and test

6. Before the initiation of model training, the establishment of our features and label columns is a prerequisite. Within the context of this dataset, the column designated as our label is identified as `isFraud`. To proceed, a variable named `features` constructed as an array including the variables:

- `amount`
- `isPayment`
- `isMovement`
- `accountDiff`


In [12]:
# Create features and label variables
features = transactions[['amount','isPayment','isMovement','accountDiff']]
label = transactions['isFraud']

7. Split the data into training and test sets using sklearn‘s `train_test_split()` method.

In [13]:
# Split dataset
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    label, 
                                                    test_size=0.3)

#### Data Normalization

8. Because sklearn's Logistic Regression implementation incorporates Regularization, it is essential to scale our feature data. This has been achieved using the StandardScaler object. The process includes applying `.fit_transform()` to the training features and later using `.transform()` on the test features.

In [14]:
# Normalize the features variables
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

9. A `LogisticRegression` model is generated using sklearn and is fitted onto the training data through the use of `.fit()`.
 The initial threshold used is the default value of 0.5.

In [15]:
# Fit the model to the training data
model = LogisticRegression()
fraud_lr = model.fit(X_train, y_train)

10. The model's `.score()` method is executed on the train and test data, and their relative score is then displayed.

By scoring the model with the training data, the trained model evaluates the transactions and predicts which ones are fraudulent. The resulting score represents the accuracy, or the percentage of accurate classifications.

In [17]:
# Save and print the predicted outcomes
y_pred = fraud_lr.predict(X_test)

In [19]:
# Print out the confusion matrix
from sklearn.metrics import confusion_matrix
print('confusion matrix: ')
print(confusion_matrix(y_test, y_pred))

# Print F1 score here:
from sklearn.metrics import f1_score
print('f1 score :', f1_score(y_test, y_pred))


confusion matrix: 
[[201   0]
 [ 50  49]]
f1 score : 0.6621621621621622


In [15]:
# Score the model on the test data
print('Train accuracy :', fraud_lr.score(X_train, y_train))
print('Test accuracy :', fraud_lr.score(X_test, y_test))

Train accuracy : 0.8428571428571429
Test accuracy : 0.8433333333333334


11. The coefficients for our model are printed to assess the significance of each feature column in the prediction process. It allows us to determine which feature holds the greatest importance and which one has the least impact in the prediction.

In [16]:
# Print the model coefficients
fraud_lr.coef_

array([[ 2.30897274, -0.63733573,  2.08183223, -0.9664791 ]])

In [37]:
features.columns

Index(['amount', 'isPayment', 'isMovement', 'accountDiff'], dtype='object')

The most impactful features based on the ranking: Amount, isMovement, isPayment, accountDif

#### Model Prediction

12. Our model will now be applied to process additional transactions that have passed through our systems. Four randomly created numpy arrays hold information about new sample transactions, labeled as "New transaction data."

In [17]:
# New transaction data
transaction1 = np.array([123456.78, 0.0, 1.0, 54670.1])
transaction2 = np.array([98765.43, 1.0, 0.0, 8524.75])
transaction3 = np.array([543678.31, 1.0, 0.0, 510025.5])
transaction4 = np.array([28178, 0, 0, 254])

# Combine new transactions into a single array
sample_transactions = np.stack((transaction1, transaction2, transaction3, transaction4))
sample_transactions

13. Considering that this Logistic Regression model was trained using scaled feature data, it is necessary to also scale the feature data for our prediction task .The `.transform()` method of `StandardScaler` object will be applied to the `sample_transactions` array.

In [24]:
# Normalize the new transactions
new_transactions_scaled = scaler.transform(sample_transactions)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


14. Fraudulent transactions are determined by utilizing the .predict() method of the model on the `new_transactions_scaled` array. The results are then printed for observation.

To view the probabilities behind these predictions, the model's `.predict_proba()` method has been chosen. The 1st column signifies the probability of a non-fraudulent transaction, while the 2nd column indicates the probability of a fraudulent transaction—calculated by the model for the classification decision. With the threshold set at 0.5, none of the new transactions have been categorized as fraudulent transactions.

In [28]:
# Predict fraud on the new transactions
print(model.predict(new_transactions_scaled))
print(model.predict_proba(new_transactions_scaled))
# Show probabilities on the new transactions

[0 0 0 0]
[[0.60792627 0.39207373]
 [0.99803366 0.00196634]
 [0.99644741 0.00355259]
 [0.99201883 0.00798117]]
